### Objective

### Installation and Setup

### Twitter API

# Lets create twitter object and use its API. Code snippets below will show you how to use this API

In [2]:
from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRestPager

ImportError: No module named TwitterAPI

In [ ]:
o = TwitterOAuth.read_file('credentials.txt')

In [ ]:
o.access_token_key

In [ ]:
# Using OAuth1...
twitter = TwitterAPI(o.consumer_key,
                 o.consumer_secret,
                 o.access_token_key,
                 o.access_token_secret)

In [ ]:
help(twitter)

In [ ]:
# What can we do with this twitter object?
# builtin method `dir` tells us...
dir(twitter)

In [ ]:
twitter.auth

In [ ]:
# Get help on the `request` method using the builtin method called...`help`
help(twitter.request)

In [ ]:
# Let's start by querying the search API
response = twitter.request('search/tweets', {'q': 'big+data'}) 

In [ ]:
# What object is returned?
# builtin type method will tell us.
print type(response)
dir(response)

In [ ]:
response.json

In [ ]:
response.status_code
# See https://dev.twitter.com/overview/api/response-codes

In [ ]:
tweets = [r for r in response]

In [ ]:
print('found %d tweets' % len(tweets))

In [ ]:
type(tweets)

In [ ]:
type(tweets[0])

In [ ]:
tweets[0]

In [ ]:
help(tweets[0])

In [ ]:
tweets[0].keys()

In [ ]:
tweets[0]['text']

In [ ]:
tweets[0]['created_at']

In [ ]:
tweets[0]['text']

In [ ]:
tweets[0]['user']

In [ ]:
user = tweets[0]['user']
print('screen_name=%s, name=%s, location=%s' % (user['screen_name'], user['name'], user['location']))

In [ ]:
# Who follows this person?
# https://dev.twitter.com/docs/api/1.1/get/followers/list
screen_name = user['screen_name']
response  = twitter.request('followers/list', {'screen_name': screen_name, 'count':200})
followers = [follower for follower in response]
        
print 'found %d followers for %s' % (len(followers), screen_name)
# See more about paging here: https://dev.twitter.com/docs/working-with-timelines

In [ ]:
print followers[0]['screen_name']

##  Limitations:       Can only search 2 weeks in past         But can get up to 3,200 most recent tweets of a user     Rate limits! https://dev.twitter.com/docs/rate-limiting/1.1/limits         e.g., 180 requests in 15 minute window 

# Get BestBuy timeline for the deals screen-name

# This is the screen name for BestBuy_Deals

In [ ]:
# Get BestBuyDeals timeline = ''
screen_name = 'BestBuy_Deals'
timeline = [tweet for tweet in twitter.request('statuses/user_timeline',
                                                {'screen_name': screen_name,
                                                 'count': 200})]
print 'got %d tweets for user %s' % (len(timeline), screen_name)

In [ ]:
# Print time got created.

timeline[3]['created_at']

In [ ]:
# Print the text.
print '\n\n\n'.join(t['text'] for t in timeline)

In [ ]:
# Count words
from collections import Counter  # This is just a fancy dict mapping from object->int, starting at 0.
counts = Counter()
for tweet in timeline:
    counts.update(tweet['text'].lower().split())
print('found %d unique terms in %d tweets' % (len(counts), len(timeline)))
counts.most_common(10)

In [ ]:
import re
for tweet in timeline:
    deal = tweet['text']
    print deal + '\n'


# Find  the deals in the BestBuy_Deal tweets  that match products in SmartPortables MySQL product table

In [ ]:
import re
import pymysql

cnx = pymysql.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='exampledatabase')
cursor = cnx.cursor()

query = ("SELECT id FROM ProductDetails")
cursor.execute(query)


dealMatchGauranteed=[]
for product in cursor:
    for tweet in timeline:
        deal = (tweet['text']).encode('ascii','ignore')
        if (len(re.findall(r'\s'+product[0]+'\s',deal)) >= 1):
            dealMatchGauranteed = dealMatchGauranteed + [deal]
        


In [ ]:
# Sanity Test that  we got some deals
dealMatchGauranteed[:5]

# Create and write the deals into DealMatches.txt file that will be used by web-app of SmartPortables to display two deal matches

In [ ]:
dealMatchFile = open('DealMatches.txt', 'w')

for deal in dealMatchGauranteed:
  dealMatchFile.write("%s\n" % deal)

dealMatchFile.close()